This is a example for customer classification. The principle is the RFM model (recency, frequency, monetary). With cluster algorism, it is very easy to classify the customer into different groups (Hero, growing hero, lapsing hero, lost hero, Loyalty customer, new tester, lapsing, and lost casual.)
After identify the class of the customer, company can easily set up a marketing strategy to provide efficent customer services.

First, we need to explore the data we have. In this example, the data is about a airline complany CRM records, 23 columns are given. We have a relatively clean data, some features have been generated by the CRM itself. We inport the data, and explore the quality of the data.

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
#os.getcwd()
flt = pd.read_csv('../input/flight_ENG_2.csv')
print(flt.shape)
flt.head()

Next, we need to check the missing values.

In [ ]:
missing = flt.shape[0] - flt.count()
print(missing[missing != 0])

We find that the most missing values are in the location columns, these information has very limited function for clustering, we ignore them. However, SUM_YR presents the ticket income, the total missing is 689, a low precentage of the number of records, we drop them. And creat histograms to check the KEY columns.

In [ ]:
flt['SUM_YR'] = flt['SUM_YR_1'] + flt['SUM_YR_2']
def hist_list(lst):
    plt.figure(figsize=[len(lst)*3,5])
    i = 1
    for col in lst:
        ax = plt.subplot(1,len(lst),i)
        ax.hist(flt[col].dropna(),50)
        plt.title(col)
        i = i+1
key_columns = ['SUM_YR','FLIGHT_COUNT','LAST_TO_END','SEG_KM_SUM']
hist_list(key_columns)
print (flt[key_columns].describe())

In [ ]:
flt['R'] = -scale(np.log(flt['LAST_TO_END']))
flt['F'] = scale(np.log(flt['FLIGHT_COUNT']))
flt['M'] = scale(np.log(flt['SEG_KM_SUM']))
RFM = ['R','F','M']
hist_list(RFM)


In [ ]:
km = KMeans(n_clusters=6, max_iter=500)
km.fit(flt[RFM])
flt['cluster']=km.labels_
ax = plt.subplot(111,projection='3d')
ax.scatter(flt['R'],flt['F'],flt['M'], 
           c=flt['cluster'])
ax.set_xlabel('R')
ax.set_ylabel('F')
ax.set_zlabel('M')
plt.show()

In [ ]:
def cluster_bar(names,grp='cluster'):
    a = len(names)
    i = 1
    cl = ['bgrcmykw'[c] for c in range(len('bgrcmykw'))]
    plt.figure(figsize=(8,4))
    plt.subplots_adjust(wspace=3)

    for col in names:
        ax = plt.subplot(1,a,i)
        g = flt.groupby(grp)
        x = g[col].mean().index
        y = g[col].mean().values
        ax.barh(x,y,color=cl[i-1])
        plt.title(col)
        i = i+1
        
cluster_bar(['LAST_TO_END','FLIGHT_COUNT','SEG_KM_SUM'])

In [ ]:
cluster_name = {0:'lost_casual',1:'lapsing hero',2:'new tester',
                3:'hero',4:'lapsing customer',5:'growing hero'}
flt['classification'] = flt['cluster'].map(cluster_name)
criteria = ['AGE','SUM_YR','Points_Sum','avg_discount']
cluster_bar(criteria,'classification')